In [5]:
import os
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm
# Define paths
image_folder = "/data/home/bowen/projects/prairie_dog_2/data/old_data/two_classes/run1_run2_yolo/images/run1_tiles"
gt_folder = "/data/home/bowen/projects/prairie_dog_2/data/old_data/two_classes/run1_run2_yolo/labels/run1_tiles"
pred_folder = "/data/home/bowen/projects/prairie_dog_2/yolov5_a100/runs/val/run12_con_1000epoch10/labels"
output_folder = "train/output_visualizations"

os.makedirs(output_folder, exist_ok=True)

In [6]:
# YOLO label format: class_id center_x center_y width height (normalized)
def read_yolo_labels(txt_path):
    """Reads YOLO format labels from a text file."""
    labels = []
    if os.path.exists(txt_path):
        with open(txt_path, "r") as f:
            for line in f.readlines():
                parts = line.strip().split()
                if len(parts) >= 5:  # At least class and bbox info
                    class_id = int(parts[0])
                    bbox = list(map(float, parts[1:5]))  # center_x, center_y, width, height
                    confidence = float(parts[5]) if len(parts) == 6 else None
                    labels.append((class_id, bbox, confidence))
    return labels

# Draw bounding boxes
def draw_boxes(image, labels, color, label_text=""):
    """Draws bounding boxes on the image."""
    h, w, _ = image.shape
    for class_id, bbox, confidence in labels:
        center_x, center_y, width, height = bbox
        x1 = int((center_x - width / 2) * w)
        y1 = int((center_y - height / 2) * h)
        x2 = int((center_x + width / 2) * w)
        y2 = int((center_y + height / 2) * h)

        # Draw rectangle
        cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)

        # Display label and confidence if available
        label_str = f"{label_text}{class_id}"
        if confidence is not None:
            label_str += f" ({confidence:.2f})"
        cv2.putText(image, label_str, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

# Process images
for img_file in tqdm(os.listdir(image_folder)):
    if img_file.endswith((".jpg", ".png", ".jpeg")):
        img_path = os.path.join(image_folder, img_file)
        gt_path = os.path.join(gt_folder, img_file.rsplit(".", 1)[0] + ".txt")
        pred_path = os.path.join(pred_folder, img_file.rsplit(".", 1)[0] + ".txt")
        
        # Read labels and predictions
        gt_labels = read_yolo_labels(gt_path)
        pred_labels = read_yolo_labels(pred_path)

        # Skip images without ground truth and predictions
        if not gt_labels and not pred_labels:
            continue
        
        # Read image
        image = cv2.imread(img_path)
        if image is None:
            continue
        
        # Draw ground truth (green) and predictions (red)
        draw_boxes(image, gt_labels, (0, 255, 0), "GT: ")
        draw_boxes(image, pred_labels, (0, 0, 255), "Pred: ")
        
        # Save the image with plotted labels
        save_path = os.path.join(output_folder, img_file)
        cv2.imwrite(save_path, image)

print("Processing complete. Check the output_visualizations folder.")

100%|██████████| 7616/7616 [00:33<00:00, 225.78it/s]

Processing complete. Check the output_visualizations folder.


In [7]:
import os
import cv2
output_folder_both = "train/output_visualizations"  # Both GT & Pred
output_folder_gt = "train/output_only_gt"  # GT but no Pred
output_folder_pred = "train/output_only_pred"  # Pred but no GT

# Ensure output folders exist
os.makedirs(output_folder_both, exist_ok=True)
os.makedirs(output_folder_gt, exist_ok=True)
os.makedirs(output_folder_pred, exist_ok=True)

# YOLO label format: class_id center_x center_y width height (normalized)
def read_yolo_labels(txt_path):
    """Reads YOLO format labels from a text file."""
    labels = []
    if os.path.exists(txt_path):
        with open(txt_path, "r") as f:
            for line in f.readlines():
                parts = line.strip().split()
                if len(parts) >= 5:  # At least class and bbox info
                    class_id = int(parts[0])
                    bbox = list(map(float, parts[1:5]))  # center_x, center_y, width, height
                    confidence = float(parts[5]) if len(parts) == 6 else None
                    labels.append((class_id, bbox, confidence))
    return labels

# Draw bounding boxes
def draw_boxes(image, labels, color, label_text=""):
    """Draws bounding boxes on the image."""
    h, w, _ = image.shape
    for class_id, bbox, confidence in labels:
        center_x, center_y, width, height = bbox
        x1 = int((center_x - width / 2) * w)
        y1 = int((center_y - height / 2) * h)
        x2 = int((center_x + width / 2) * w)
        y2 = int((center_y + height / 2) * h)

        # Draw rectangle
        cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)

        # Display label and confidence if available
        label_str = f"{label_text}{class_id}"
        if confidence is not None:
            label_str += f" ({confidence:.2f})"
        cv2.putText(image, label_str, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

# Process images
for img_file in tqdm(os.listdir(image_folder)):
    if img_file.endswith((".jpg", ".png", ".jpeg")):
        img_path = os.path.join(image_folder, img_file)
        gt_path = os.path.join(gt_folder, img_file.rsplit(".", 1)[0] + ".txt")
        pred_path = os.path.join(pred_folder, img_file.rsplit(".", 1)[0] + ".txt")
        
        # Read labels and predictions
        gt_labels = read_yolo_labels(gt_path)
        pred_labels = read_yolo_labels(pred_path)

        # Skip images that have neither GT nor Predictions
        if not gt_labels and not pred_labels:
            continue
        
        # Read image
        image = cv2.imread(img_path)
        if image is None:
            continue
        
        # Draw bounding boxes
        if gt_labels:
            draw_boxes(image, gt_labels, (0, 255, 0), "GT: ")  # Green for GT
        if pred_labels:
            draw_boxes(image, pred_labels, (0, 0, 255), "Pred: ")  # Red for Predictions

        if gt_labels and not pred_labels:
            save_path = os.path.join(output_folder_gt, img_file)  # Only GT, No Pred
        elif pred_labels and not gt_labels:
            save_path = os.path.join(output_folder_pred, img_file)  # Only Pred, No GT
        else:
            continue  # Should never reach here due to earlier condition

        # Save the annotated image
        cv2.imwrite(save_path, image)

print("Processing complete. Check the output folders:")
print(f"- {output_folder_both}: Contains images with both GT and predictions")
print(f"- {output_folder_gt}: Contains images with only GT (no predictions)")
print(f"- {output_folder_pred}: Contains images with only predictions (no GT)")


100%|██████████| 7616/7616 [00:10<00:00, 761.18it/s]

Processing complete. Check the output folders:
- train/output_visualizations: Contains images with both GT and predictions
- train/output_only_gt: Contains images with only GT (no predictions)
- train/output_only_pred: Contains images with only predictions (no GT)
